Tip: Use a CPU engine while writing the code and once you know the code works, use a GPU engine to train the model. (Runtime -> Change runtime type). This is because you should not ask for a GPU if you do not need it (you only need it when training a model).

## DISCLAIMER
We were not able to train our model, as we were stuck with an error when iterating over the train_dataloader in the training loop:

```
Output exceeds the size limit. Open the full output data in a text editor
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
/Users/me/git/dl4nlp/Homework_6/Copy_of_DL4NLP_QA_HW6.ipynb Cell 39 in <cell line: 9>()
      8 model.train()
      9 for epoch in range(num_epochs):
---> 10     for batch in train_dataloader:
     11         outputs = model(**batch)
     12         list_loss.append(outputs.loss)

File ~/git/dl4nlp/venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:652, in _BaseDataLoaderIter.__next__(self)
    649 if self._sampler_iter is None:
    650     # TODO(https://github.com/pytorch/pytorch/issues/76750)
    651     self._reset()  # type: ignore[call-arg]
--> 652 data = self._next_data()
    653 self._num_yielded += 1
    654 if self._dataset_kind == _DatasetKind.Iterable and \
    655         self._IterableDataset_len_called is not None and \
    656         self._num_yielded > self._IterableDataset_len_called:

File ~/git/dl4nlp/venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:692, in _SingleProcessDataLoaderIter._next_data(self)
    690 def _next_data(self):
    691     index = self._next_index()  # may raise StopIteration
--> 692     data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
    693     if self._pin_memory:
    694         data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)
...
    129         else:
--> 130             batch[k] = torch.tensor([f[k] for f in features])
    132 return batch

TypeError: only integer tensors of a single element can be converted to an index
```
We tried to convert the respective tensor to int, but that did not fix the issue.

## HOWEVER
We tried to write all relevant code for the evaluation (without being able to run it).
If possible, we would be very happy to receive partial points for task 1.5, especially since the last two tasks are not possible for us to finish :/

# Install the Libraries

In [1]:
!pip install transformers
!pip install datasets
!pip install huggingface_hub
!pip install evaluate


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 1.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# model_name = "microsoft/xtremedistil-l6-h256-uncased"
# model_name = "microsoft/xtremedistil-l12-h384-uncased"
# model_name = "microsoft/xtremedistil-l6-h384-uncased"
# model_name = "distilbert-base-uncased"
# model_name = "microsoft/MiniLM-L12-H384-uncased"
model_name = "huawei-noah/TinyBERT_General_4L_312D"
# model_name = "huawei-noah/TinyBERT_General_6L_768D"

# 1. Data and Model Preparation 

## 1.1. Set the seed
This will ensure the reproducibility of your code in other devices

In [3]:
import torch
torch.manual_seed(0) # set manual seed for reproducibility

## 1.2 Load the Dataset
We will use the the well-known MRQA 2019 Shared Task in this homework. Download your assigned dataset from https://github.com/mrqa/MRQA-Shared-Task-2019. Hint, you can use
```
!wget https://....
```
 to download it directly into your colab instance. Please note that the data is not stored across sessions.

You should use your the training set to train your model and the dev set to evaluate it. 

**Do not use the dev set in the training phase!**

In [6]:
# download your dataset
!curl -o train_TriviaQA-web.jsonl.gz https://s3.us-east-2.amazonaws.com/mrqa/release/v2/train/TriviaQA-web.jsonl.gz
!curl -o dev_TriviaQA-web.jsonl.gz https://s3.us-east-2.amazonaws.com/mrqa/release/v2/dev/TriviaQA-web.jsonl.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  11.5M      0  0:00:29  0:00:29 --:--:-- 12.0M00:04  0:00:41 7745k9 --:--:-- 12.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42.8M  100 42.8M    0     0  8712k      0  0:00:05  0:00:05 --:--:-- 9458k  0:00:06  0:00:03  0:00:03 6938k


In [140]:
# convert dataset into the same format as squad
import json
import gzip
from tqdm import tqdm

# creates one example per answer
def convert_train_dataset(file_in, file_out):

  for line in tqdm(file_in):
    example = json.loads(line)

    # skip header
    if 'header' in example:
      continue

    context = example['context']
    for qa in example['qas']:
      id = qa['id']
      question = qa['question']
      

      # if len(qa['detected_answers']) == 0:
      #   print("Could not find any answer for id={id}")
      #   continue

      for answer in qa['detected_answers']:
        answers = {
          'text': [],
          'answer_start': []
        }

        answers['text'].append(answer['text'])
        answers['answer_start'].append(answer['char_spans'][0][0])

        new_example = json.dumps({
          'id': id,
          'context': context,
          'question': question,
          'answers': answers
        })

      file_out.write(f'{new_example}\n')


# creates one example per question
def convert_dev_dataset(file_in, file_out):

  for line in tqdm(file_in):
    example = json.loads(line)

    # skip header
    if 'header' in example:
      continue

    context = example['context']
    for qa in example['qas']:
      id = qa['id']
      question = qa['question']
      
      answers = {
        'text': [],
        'answer_start': []
      }
      
      # if len(qa['detected_answers']) == 0:
      #   print("Could not find any answer for id={id}")
      #   continue

      for answer in qa['detected_answers']:
        answers['text'].append(answer['text'])
        answers['answer_start'].append(answer['char_spans'][0][0])

      new_example = json.dumps({
        'id': id,
        'context': context,
        'question': question,
        'answers': answers
      })

      file_out.write(f'{new_example}\n')


file_in = gzip.open('train_TriviaQA-web.jsonl.gz', 'rb')
file_out = open(f'train_TriviaQA-web.jsonl.gz_converted.jsonl', 'w')

convert_train_dataset(file_in, file_out)

file_in = gzip.open('dev_TriviaQA-web.jsonl.gz', 'rb')
file_out = open(f'dev_TriviaQA-web.jsonl.gz_converted.jsonl', 'w')

convert_dev_dataset(file_in, file_out)


61689it [00:22, 2710.06it/s]
7786it [00:02, 2786.82it/s]


In [146]:
# loading the dataset
from datasets import load_dataset

dataset = load_dataset('json', data_files={
  'train': 'train_TriviaQA-web.jsonl.gz_converted.jsonl',
  'dev': 'dev_TriviaQA-web.jsonl.gz_converted.jsonl'
})

Using custom data configuration default-e9657f815bc7eb5d


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /Users/me/.cache/huggingface/datasets/json/default-e9657f815bc7eb5d/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## 1.3 Load the Model

In [9]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'fit_denses.0.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'fit_denses.1.weight', 'fit_denses.0.weight', 'fit_denses.2.weight', 'fit_denses.4.weight', 'fit_denses.4.bias', 'fit_denses.3.bias', 'fit_denses.1.bias', 'fit_denses.2.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.3.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you

## 1.3 Load the Tokenizer
Intantiate a tokenizer for your model. Check in the **quicktour** of the **Transformer** **library** for more help https://huggingface.co/docs/transformers/quicktour 

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Data Preprocessing


## 2.1 Data Exploration
Before jumping into machine learning code, do some data exploration. To train an extractive-QA model we need to use the `question`, `context`/`passage`, and the `answer`. Explore the dataset to learn how to extract these three pieces of information. You should see the first three data instances and print their questions, contexts/passages, and answers.

In [173]:
# data exploration
for idx in range(3):
  print(f'Example {idx}:')
  print('\tContext (capped at 100 chars): ', dataset['train'][idx]["context"][:100])
  print('\tQuestion: ', dataset['train'][idx]["question"])
  print('\tAnswer: ', dataset['train'][idx]["answers"]['text'][0])

Example 0:
	Context (capped at 100 chars):  [DOC] [TLE] THEME FROM MAHOGANY - (DO YOU KNOW WHERE YOU'RE GOING TO ...THEME FROM MAHOGANY - (DO YO
	Question:  Do You Know Where You're Going To? was the theme from which film?
	Answer:  mahogany
Example 1:
	Context (capped at 100 chars):  [DOC] [TLE] Breed Breakdown: Doberman Pinscher - unleashmagazine.comBreed Breakdown: Doberman Pinsch
	Question:  Which actor had a Doberman Pinscher called Kirk?
	Answer:  william shatner
Example 2:
	Context (capped at 100 chars):  [DOC] [TLE] Maurice Chevalier - Thank Heaven for Little Girls (from ...Maurice Chevalier - Thank Hea
	Question:  Which musical featured the song Thank Heaven for Little Girls?
	Answer:  gigi


## 2.2 Dataset Encoding
The model should take a question and a context to return an answer. However, we cannot input strings into a Transformer encoder. First, we need to tokenize and encode the text into a format that the model understand. 

The input of most Transformers models are `input_ids`, `token_type_ids`, and `attention_mask`. You should use the tokenizer to obtain this. Note, distilbert does not use `token_type_ids`.

Transform one pair of question-context into features (`input_ids`, `token_type_ids`, and `attention_mask`)

In [29]:
features = tokenizer(
  dataset["train"][0]["question"],
  dataset["train"][0]["context"],
  max_length=512,
  truncation="only_second",
  return_offsets_mapping=True,
  padding="max_length",
)
features

{'input_ids': [101, 2079, 2017, 2113, 2073, 2017, 1005, 2128, 2183, 2000, 1029, 2001, 1996, 4323, 2013, 2029, 2143, 1029, 102, 1031, 9986, 1033, 1031, 1056, 2571, 1033, 4323, 2013, 23867, 1011, 1006, 2079, 2017, 2113, 2073, 2017, 1005, 2128, 2183, 2000, 1012, 1012, 1012, 4323, 2013, 23867, 1011, 1006, 2079, 2017, 2113, 2073, 2017, 1005, 2128, 2183, 2000, 1007, 1011, 7858, 1031, 11968, 1033, 4323, 2013, 23867, 1011, 1006, 2079, 2017, 2113, 2073, 2017, 1005, 2128, 2183, 2000, 1007, 1031, 11968, 1033, 2215, 2000, 3422, 2023, 2153, 2101, 1029, 1031, 11968, 1033, 3696, 1999, 2000, 5587, 2023, 2678, 2000, 1037, 2377, 9863, 1012, 1031, 11968, 1033, 2342, 2000, 3189, 1996, 2678, 1029, 1031, 11968, 1033, 3696, 1999, 2000, 3189, 15884, 4180, 1012, 1031, 11968, 1033, 5790, 2003, 2800, 2043, 1996, 2678, 2038, 2042, 12524, 1012, 1031, 11968, 1033, 2023, 3444, 2003, 2025, 2800, 2157, 2085, 1012, 3531, 3046, 2153, 2101, 1012, 1031, 11968, 1033, 2405, 2006, 12022, 2654, 1010, 2286, 1031, 11968, 1033, 

Next, we need to create the labels for the model. Since we are creating an extractive model, the labels are the starting and ending indexes. This is not straightforward, so we will use part of HuggingFace's code. This method uses `offset_mapping`, this maps wordpiece indexes with the starting and ending character indexes. Let's see an example to understand its purpouse.

* wordpiece tokens: ['to', 'whom']
* input_ids: tensor([2000, 3183])
* offset_mapping: tensor([[0, 2], [3, 7]])

As you can see input_text[0:2] == 'to' and input_text[3:7] == 'whom'

In [174]:
def get_labels(examples, features):
    '''
    Input:
        - examples:
        - features: output of the tokenizer. It should include the offset_maping.
        Make sure to get this value when you tokenize your data
    Output:
        - start_positions: list of wordpiece idx where the answer starts
        - end_positions: list of wordpiece idx where the answer ends
    '''
    offset_mapping = features["offset_mapping"]
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = features.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
    return (start_positions, end_positions)

Now, let's make a function `preprocess_function` that takes a list of examples and returns the features :
- `input_ids`
- `token_type_ids` (if needed, depends on the model)
- `attention_mask`
- `start_positions`
- `end_positions`

In [107]:
def preprocess_function(examples):
    '''
    Input:
        - list of QA examples
    Output:
        - dictionary of features
    '''

    # First, let's remove duplicated spaces from the question. This occurrs in some datasets and may create problems.
    questions = [q.strip() for q in examples["question"]]

    # Get the features
    features = tokenizer(
      questions,
      examples["context"],
      max_length=512,
      truncation="only_second",
      return_offsets_mapping=True,
      padding="max_length",
    )
    # Get the labels
    (start_positions, end_positions) = get_labels(examples, features)

    # add the start and end positions to the dictionary of features
    features['start_positions'] = start_positions
    features['end_positions'] = end_positions

    # Return the features
    return features

Let's check that the function works as expected

In [108]:
preprocess_function(dataset['train'][0:2])

{'input_ids': [[101, 2079, 2017, 2113, 2073, 2017, 1005, 2128, 2183, 2000, 1029, 2001, 1996, 4323, 2013, 2029, 2143, 1029, 102, 1031, 9986, 1033, 1031, 1056, 2571, 1033, 4323, 2013, 23867, 1011, 1006, 2079, 2017, 2113, 2073, 2017, 1005, 2128, 2183, 2000, 1012, 1012, 1012, 4323, 2013, 23867, 1011, 1006, 2079, 2017, 2113, 2073, 2017, 1005, 2128, 2183, 2000, 1007, 1011, 7858, 1031, 11968, 1033, 4323, 2013, 23867, 1011, 1006, 2079, 2017, 2113, 2073, 2017, 1005, 2128, 2183, 2000, 1007, 1031, 11968, 1033, 2215, 2000, 3422, 2023, 2153, 2101, 1029, 1031, 11968, 1033, 3696, 1999, 2000, 5587, 2023, 2678, 2000, 1037, 2377, 9863, 1012, 1031, 11968, 1033, 2342, 2000, 3189, 1996, 2678, 1029, 1031, 11968, 1033, 3696, 1999, 2000, 3189, 15884, 4180, 1012, 1031, 11968, 1033, 5790, 2003, 2800, 2043, 1996, 2678, 2038, 2042, 12524, 1012, 1031, 11968, 1033, 2023, 3444, 2003, 2025, 2800, 2157, 2085, 1012, 3531, 3046, 2153, 2101, 1012, 1031, 11968, 1033, 2405, 2006, 12022, 2654, 1010, 2286, 1031, 11968, 1033,

You should see something similar to this:

```
{'input_ids': [[101, 2000, 3183, ...], [101, 2054, 2003, ...]], 'token_type_ids': [[0, 0, 0, 0, ..., 1, 1, 1, 1, 1, 1,...], [0, 0, 0,...,  1, 1, 1, 1, ...]], 'attention_mask': [[1, 1, 1, ..., 0, 0, 0], [1, 1, 1, 1, 1,... 0, 0, 0]], 'start_positions': [130, 52], 'end_positions': [137, 56]}


```

You should apply this function to every instance of the dataset.
You could do this with a for loop, but there are more efficient ways to do it.
You could use a *map* operation (feel free to use the method you prefer)

https://huggingface.co/docs/datasets/process#map

In [176]:
features = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Now that you have the features, you will need to create dataloaders for the training process. Hint: the method `set_format` from the Dataset class might be useful

In [177]:
features = dataset.map(preprocess_function, batched=True)

from torch.utils.data import DataLoader
from transformers import default_data_collator


# remove text columns from features
features = features.flatten()
#features = features.rename_column('answers.answer_start', 'answer_start')
features = features.remove_columns(['id', 'context', 'question', 'answers.text', 'start_positions', 'end_positions'])

# also remove offset mappings from dev
features['dev'] = features['dev'].remove_columns(["offset_mapping"])

features.set_format("torch")

# explicitly convert tensors to integer
for i in range(len(features['train'])):
    features['train'][i]['answers.answer_start'] = features['train'][i]['answers.answer_start'].type(torch.uint8)
for i in range(len(features['dev'])):
    features['dev'][i]['answers.answer_start'] = features['dev'][i]['answers.answer_start'].type(torch.uint8)

features = features.remove_columns(['answers.answer_start'])

train_dataloader = DataLoader(
    features['train'], 
    #shuffle=True,
    collate_fn=default_data_collator, 
    batch_size=16
)

dev_dataloader = DataLoader(
    features['dev'], 
    collate_fn=default_data_collator, 
    batch_size=32
    # no shuffling in dev!
)



# trying to debug  
#print(features['train'][0]['offset_mapping'])
#print(features['train'])
#print(features['train'][0])

#for idx in range(10):
#    print(f"##### idx={idx} #####")
#    for k, v in features['train'][idx].items():
#        if k == "answers.answer_start":
#            print(k, len(v), v.dtype)

#for batch in train_dataloader: -> TypeError: only integer tensors of a single element can be converted to an index
#    print(batch)
#    break

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# 3. Training

Congratulations! You have the data ready to train. Obtaining and processing the data probably is the most important part in most machine learning aplications. Now, we can train the model.

We will train the model for the first time with the default hyperparameters:
- learning rate: 1e-5 (the default when fine-tuning a Transformer model)
- train batch size: 16
- evaluation batch size: 32
- epochs: 1
- weight_decay: 0.01
- optimizer: Adam (https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.AdamW)
- linear scheduler with 0 warmup steps (https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.SchedulerType)

In [139]:
# HP config 1 (original)
from torch.optim import AdamW
from transformers import get_scheduler
import torch

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters(), weight_decay=0.01, lr=1e-5)
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


In [ ]:
# HP config 2 (lower weight decay, lower lr)
from torch.optim import AdamW
from transformers import get_scheduler
import torch

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters(), weight_decay=0.001, lr=1e-6)
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [ ]:
# HP config 3 (more epochs, higher lr, two warm-up steps)
from torch.optim import AdamW
from transformers import get_scheduler
import torch

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters(), weight_decay=0.01, lr=5e-5)
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=2, num_training_steps=num_training_steps)

In [175]:
# HP config 4 (more epochs, more warm-up steps, lower weight decay, lower lr)
from torch.optim import AdamW
from transformers import get_scheduler
import torch

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters(), weight_decay=0.001, lr=1e-6)
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=2, num_training_steps=num_training_steps)

We will evaluate the model at the end of each epoch, so we need to define an evaluation method. However, the outputs of the model are logits. What we want are the answers in strings. ***Postprocessing*** this is not straightforward, so we will give you the skeleton of the method you need.

As for the metrics, in extractive QA, the most common metrics are EM and F1 a.k.a. squad-metric. The dataset library has already implemented most NLP metrics, including the squad-metric.

You can calculate the performance of the model with:

```
squad_metric.compute(predictions=predictions, references=references)
```

The input must be as in the example below:
```
predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22'}]
references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
```



In [20]:
from tqdm.auto import tqdm
import numpy as np
from evaluate import load
squad_metric = load("squad")

def evaluation(model, dev_dataloader):
    list_outputs = []
    # get raw predictions
    for batch in dev_dataloader:
        with torch.no_grad():
            list_outputs.append(model(**batch))

    # post-process the predictions to get strings
    predictions = postprocess_qa_predictions(dataset['dev'], features['dev'], raw_predictions=list_outputs)
    # compute EM and F1

    well_formed_predictions = []
    for id, valid_answers in predictions.items():
      
      for valid_answer in valid_answers:
        well_formed_predictions.append({
            'prediction_text': valid_answer['text'],
            'id': id
        })

    references = []
    for data_point in dataset['dev']:
      references.append({
          'answers': data_point['answers'],
          'id': data_point['id']
      })

    results = squad_metric.compute(predictions=well_formed_predictions, references=references)
    return results, predictions

def postprocess_qa_predictions(dataset, features, raw_predictions, topk=1, max_answer_length=30):
    '''
    Input:
        - dataset
        - features: list of dictionaries with input_ids, attention_masks, ...
        - raw_predictions: the logits from the model
        - topk: number of predictions per question
        - max_answer_length: maximum number of wordpiece tokens for the answer.
    Output:
        - dictionary id -> confidence score and prediction (str). Example: {'56be4db0acb8001400a502ec': [{'score': 15.633667, 'text': 'Denver Broncos'}],
    '''
    predictions = dict() # dict id -> {'score': ... , 'text': ...} We will return this

    # for each data point (a QA pair) in the dataset
    for idx, data_point in enumerate(dataset):    
        
        start_logits = raw_predictions[idx].start_logits.cpu().numpy() # get the start_logits
        end_logits = raw_predictions[idx].end_logits.cpu().numpy() # get the end_logits
        offset_mapping = features[idx]['offset_mapping'] # get the offset_mapping from the dataset features  

        # get the context of this QA pair
        context = data_point['context']

        # Gather the indices of the best 50 start/end logits:
        start_indexes = np.argsort(start_logits)[-1 : -50 - 1 : -1].tolist() # hint: make this a regular python list
        end_indexes = np.argsort(end_logits)[-1 : -50 - 1 : -1].tolist()  # hint: make this a regular python list
        valid_answers = []

        for start_index in start_indexes: # for each start_index
            for end_index in end_indexes: # for each end_index

                # Make sure that indexes are not out of bounds and the offset_mapping are not None
                if (start_index < 0 or start_index >= len(context) or end_index < 0 or end_index >= len(context) or offset_mapping == None):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                # make sure that 1) the end_index is not smaller than the start index and 2) the answer length is smaller than the max_answer_length
                if end_index < start_index or (end_index - start_index) >= max_answer_length:
                    continue
                # if start_index is smaller than end_index
                if start_index < end_index: 
                    # get the char starting and ending index (hint: use the offset_mapping)
                    start_char = offset_mapping[0]
                    end_char = offset_mapping[1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index], # the sum of the starting and ending logits
                            "text": context[offset_mapping[start_index][0] : offset_mapping[end_index][1]] # answer string. What we were looking for!!!
                        }
                    )

        # sort the answers by descending score and pick only the best topk
        valid_answers = sorted(valid_answers, lambda x: x['score'], reverse=True)[:topk]
        # add the list of valid answers to the predictions dictionary
        predictions[data_point['id']] = valid_answers
    
    return predictions

**It's training time!**

In [138]:
from tqdm.auto import tqdm

list_loss = []
list_em = []
list_f1 = []

progress_bar = tqdm(range(num_training_steps))
model.train()
all_predictions = {}

for epoch in range(num_epochs):
    for batch in train_dataloader: # -> TypeError: only integer tensors of a single element can be converted to an index
        outputs = model(**batch)
        list_loss.append(outputs.loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
    # Evaluation
    model.eval()

    result, predictions = evaluation(model, dev_dataloader)
    all_predictions.update(predictions)
    list_em.append(result['exact_match'])
    list_f1.append(result['f1'])

    print(f"after epoch {epoch}: em_on_dev={result['exact_match']}, f1_on_dev={result['f1']}")

    # switch back to training mode
    model.train()

# write all predictions to file
with open(f"predictions_{model_name}-", "w") as f:
    json.dump(all_predictions, f)

# plot the loss value against the training step
from matplotlib import pyplot as plt
# loss per training step
fig = plt.figure()
plt.plot(list_loss)
plt.xlabel('training step')
plt.ylabel('loss')
fig.savefig('training_loss_plot.pdf')

# em per epoch
fig = plt.figure()
plt.plot(list_em)
plt.xlabel('epoch')
plt.ylabel('exact_match')
fig.savefig('em_plot.pdf')

# f1 per epoch
fig = plt.figure()
plt.plot(list_f1)
plt.xlabel('epoch')
plt.ylabel('f1 score')
fig.savefig('f1_plot.pdf')

  0%|          | 0/3856 [00:00<?, ?it/s]

TypeError: only integer tensors of a single element can be converted to an index

Congratulations! You trained for first QA model 🎉 . Let's analyze how the training went by plotting the data we have:
1) Plot the loss
2) Plot the EM (if you have more than 1 epoch)
3) Plot the F1 (if you have more than 1 epoch)

# Sharing your model with the community
Hoorayy!!! 🥳 🎉 You created and evaluated a QA model. It would be a shame if you don't share it with the community... It can be super useful for others' research and you can become famous!! Some QA models have 1 million downloads (https://huggingface.co/deepset/roberta-base-squad2), this can be your model! ;) So let's upload it. Don't forget to write a brief description of you model in the model card and also its EM and F1 score! Try to add as much metadata as possible. This can be very helpful for others. You can use this as a template: https://huggingface.co/docs/hub/models-cards

In [ ]:
model.push_to_hub(...)

# Deploying your model on UKP-SQuARE
The UKP lab@ TU Darmstadt is developing a QA platform to ease the analysis, research, and development of QA models. Create an account on square.ukp-lab.de and deploy your model. You don't need to write any code for this, just the id of your model in HuggingFace's Hub. Once you have it deployed, play with your model and compare it the other publicly available models. 

You should show some screenshots in your reports comparing your model with any other available model on SQuARE.